This notebook contains the code for extracting cnn features from facial features using cnn<br>
1: Truth<br>
0: Lie <br> <br>

In [ ]:
SEQ_LEN = 20
SEED_VAL = 0

IMG_HEIGHT = 128
IMG_WIDTH = 128

# Importing libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import cv2
import os
import tensorflow as tf
import keras

from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.optimizers import Adam
from keras.layers import *
from keras.callbacks import EarlyStopping
from keras.applications import ResNet50V2

# Loading dataset

In [ ]:
FACIAL_FEATURES_PATH = "/content/drive/MyDrive/CDAC-Project(Personal)/numpy-arrays/Videos/Version 3/video-frame-facial-features(20).npz"
LABELS_PATH = "/content/drive/MyDrive/CDAC-Project(Personal)/numpy-arrays/Videos/Version 3/video-frame-labels(20).npz"

In [ ]:
features = np.load(FACIAL_FEATURES_PATH)["arr_0"]
labels = np.load(LABELS_PATH)["arr_0"]

# EDA

In [ ]:
print("Lengths:\nfacial features: {}, labels: {}".format(len(features), len(labels)))

Lengths:
facial features: 325, labels: 325


In [ ]:
print("Shapes:\nfacial features: {}, labels: {}".format(features.shape, labels.shape))

Shapes:
facial features: (325, 20, 128, 128, 3), labels: (325,)


# Train Test Split

In [ ]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.25, random_state=SEED_VAL)

In [ ]:
print("Shapes\nTrain:\nFeatures: {}, labels: {}\n\nTest:\nFeatures:{}, labels: {}".format(
    features_train.shape, labels_train.shape, features_test.shape, labels_test.shape
))

Shapes
Train:
Features: (243, 20, 128, 128, 3), labels: (243,)

Test:
Features:(82, 20, 128, 128, 3), labels: (82,)


## Value counts

In [ ]:
print("Train\n{}\n\nTest\n{}".format(pd.value_counts(labels_train), pd.value_counts(labels_test)))

Train
1    125
0    118
dtype: int64

Test
0    44
1    38
dtype: int64


# CNN Model

## Callbacks

In [ ]:
SAVED_MODEL_PATH = "/content/drive/MyDrive/CDAC-Project(Personal)/saved-models/video/Version 3/video-CNN({})".format(SEQ_LEN)

cnn_cbk1 = tf.keras.callbacks.ModelCheckpoint(filepath=SAVED_MODEL_PATH,
                                             save_weights_only=False,
                                             monitor="val_accuracy",
                                             mode="max",
                                             verbose=1,
                                             save_best_only=True)


cnn_cbk2 = tf.keras.callbacks.EarlyStopping(monitor="val_loss",
                                            patience=25,
                                            verbose=1,
                                            mode="min")

## Building Model

In [ ]:
def create_cnn_model() :
    conv_base = ResNet50V2(weights="imagenet",
                           include_top=False,
                           input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

    inp = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))

    x = conv_base(inp)
    x = MaxPooling2D((2, 2))(x)
    x = Dropout(0.20)(x)

    x = Conv2D(512, (3, 3), padding="same", activation="relu")(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(256, (3, 3), padding="same", activation="relu")(x)

    x = Flatten()(x)

    op = Dense(1, activation="sigmoid")(x)


    model = Model(inp, op)
    opt = Adam(learning_rate=0.001)
    model.compile(optimizer=opt, loss="binary_crossentropy", metrics=["accuracy"])

    return model

In [ ]:
cnn_model = create_cnn_model()

In [ ]:
cnn_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 resnet50v2 (Functional)     (None, 4, 4, 2048)        23564800  
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 2, 2, 2048)       0         
 2D)                                                             
                                                                 
 dropout (Dropout)           (None, 2, 2, 2048)        0         
                                                                 
 conv2d (Conv2D)             (None, 2, 2, 512)         9437696   
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 1, 1, 512)        0         
 2D)                                                         

## Training

In [ ]:
def get_reshaped_features(features, labels, num_frames=SEQ_LEN) :
    """
    returns reshaped features and labels for cnn model (separates sequential frames into new rows)
    """

    # Duplicating label values for each frames

    cnn_labels = list()

    for i in range(features.shape[0]) :
        for j in range(num_frames) :
            cnn_labels.append(labels[i])

    cnn_labels = np.array(cnn_labels)

    # Changing label into 1D
    reshaped_cnn_labels = np.reshape(cnn_labels, (-1, 1))

    # Reshaping features
    dim1 = features.shape[0] * features.shape[1]
    dim2 = features.shape[2]
    dim3 = features.shape[3]
    dim4 = features.shape[4]

    reshaped_cnn_features = np.reshape(features, (dim1, dim2, dim3, dim4))

    return reshaped_cnn_features, reshaped_cnn_labels

In [ ]:
cnn_features_train, cnn_labels_train = get_reshaped_features(features_train, labels_train)

In [ ]:
print(cnn_features_train.shape)

(4860, 128, 128, 3)


In [ ]:
print(cnn_labels_train.shape)

(4860, 1)


In [ ]:
history = cnn_model.fit(cnn_features_train, cnn_labels_train, epochs=200, validation_split=0.10, shuffle=True, callbacks=[cnn_cbk1, cnn_cbk2])

Epoch 1/200
137/137 [==============================] - ETA: 0s - loss: 0.7557 - accuracy: 0.5482
Epoch 1: val_accuracy improved from -inf to 0.52058, saving model to /content/drive/MyDrive/CDAC-Project(Personal)/saved-models/video/Version 3/video-CNN(20)


137/137 [==============================] - 57s 345ms/step - loss: 0.7557 - accuracy: 0.5482 - val_loss: 0.9804 - val_accuracy: 0.5206
Epoch 2/200
137/137 [==============================] - ETA: 0s - loss: 0.6783 - accuracy: 0.5565
Epoch 2: val_accuracy did not improve from 0.52058
137/137 [==============================] - 18s 133ms/step - loss: 0.6783 - accuracy: 0.5565 - val_loss: 1.5868 - val_accuracy: 0.4815
Epoch 3/200
137/137 [==============================] - ETA: 0s - loss: 0.6827 - accuracy: 0.5592
Epoch 3: val_accuracy improved from 0.52058 to 0.53086, saving model to /content/drive/MyDrive/CDAC-Project(Personal)/saved-models/video/Version 3/video-CNN(20)


137/137 [==============================] - 47s 347ms/step - loss: 0.6827 - accuracy: 0.5592 - val_loss: 0.6897 - val_accuracy: 0.5309
Epoch 4/200
137/137 [==============================] - ETA: 0s - loss: 0.6909 - accuracy: 0.5290
Epoch 4: val_accuracy did not improve from 0.53086
137/137 [==============================] - 19s 141ms/step - loss: 0.6909 - accuracy: 0.5290 - val_loss: 1.7104 - val_accuracy: 0.4218
Epoch 5/200
137/137 [==============================] - ETA: 0s - loss: 0.6776 - accuracy: 0.5793
Epoch 5: val_accuracy improved from 0.53086 to 0.54938, saving model to /content/drive/MyDrive/CDAC-Project(Personal)/saved-models/video/Version 3/video-CNN(20)


137/137 [==============================] - 43s 317ms/step - loss: 0.6776 - accuracy: 0.5793 - val_loss: 0.6900 - val_accuracy: 0.5494
Epoch 6/200
137/137 [==============================] - ETA: 0s - loss: 0.6860 - accuracy: 0.5299
Epoch 6: val_accuracy did not improve from 0.54938
137/137 [==============================] - 18s 132ms/step - loss: 0.6860 - accuracy: 0.5299 - val_loss: 0.9902 - val_accuracy: 0.4671
Epoch 7/200
137/137 [==============================] - ETA: 0s - loss: 0.6795 - accuracy: 0.5473
Epoch 7: val_accuracy improved from 0.54938 to 0.55761, saving model to /content/drive/MyDrive/CDAC-Project(Personal)/saved-models/video/Version 3/video-CNN(20)


137/137 [==============================] - 46s 334ms/step - loss: 0.6795 - accuracy: 0.5473 - val_loss: 0.7008 - val_accuracy: 0.5576
Epoch 8/200
137/137 [==============================] - ETA: 0s - loss: 0.6757 - accuracy: 0.5722
Epoch 8: val_accuracy improved from 0.55761 to 0.57202, saving model to /content/drive/MyDrive/CDAC-Project(Personal)/saved-models/video/Version 3/video-CNN(20)


137/137 [==============================] - 43s 318ms/step - loss: 0.6757 - accuracy: 0.5722 - val_loss: 0.7789 - val_accuracy: 0.5720
Epoch 9/200
137/137 [==============================] - ETA: 0s - loss: 0.6681 - accuracy: 0.5629
Epoch 9: val_accuracy did not improve from 0.57202
137/137 [==============================] - 19s 135ms/step - loss: 0.6681 - accuracy: 0.5629 - val_loss: 1.1884 - val_accuracy: 0.4588
Epoch 10/200
137/137 [==============================] - ETA: 0s - loss: 0.6697 - accuracy: 0.5679
Epoch 10: val_accuracy improved from 0.57202 to 0.60905, saving model to /content/drive/MyDrive/CDAC-Project(Personal)/saved-models/video/Version 3/video-CNN(20)


137/137 [==============================] - 52s 378ms/step - loss: 0.6697 - accuracy: 0.5679 - val_loss: 0.6627 - val_accuracy: 0.6091
Epoch 11/200
137/137 [==============================] - ETA: 0s - loss: 0.6781 - accuracy: 0.5510
Epoch 11: val_accuracy did not improve from 0.60905
137/137 [==============================] - 19s 135ms/step - loss: 0.6781 - accuracy: 0.5510 - val_loss: 0.6826 - val_accuracy: 0.5885
Epoch 12/200
137/137 [==============================] - ETA: 0s - loss: 0.6833 - accuracy: 0.5412
Epoch 12: val_accuracy improved from 0.60905 to 0.63169, saving model to /content/drive/MyDrive/CDAC-Project(Personal)/saved-models/video/Version 3/video-CNN(20)


137/137 [==============================] - 44s 319ms/step - loss: 0.6833 - accuracy: 0.5412 - val_loss: 0.6778 - val_accuracy: 0.6317
Epoch 13/200
137/137 [==============================] - ETA: 0s - loss: 0.6778 - accuracy: 0.5505
Epoch 13: val_accuracy did not improve from 0.63169
137/137 [==============================] - 19s 135ms/step - loss: 0.6778 - accuracy: 0.5505 - val_loss: 0.6892 - val_accuracy: 0.5473
Epoch 14/200
137/137 [==============================] - ETA: 0s - loss: 0.6729 - accuracy: 0.5594
Epoch 14: val_accuracy did not improve from 0.63169
137/137 [==============================] - 18s 133ms/step - loss: 0.6729 - accuracy: 0.5594 - val_loss: 0.6947 - val_accuracy: 0.5556
Epoch 15/200
137/137 [==============================] - ETA: 0s - loss: 0.6627 - accuracy: 0.5750
Epoch 15: val_accuracy did not improve from 0.63169
137/137 [==============================] - 19s 136ms/step - loss: 0.6627 - accuracy: 0.5750 - val_loss: 0.6547 - val_accuracy: 0.6152
Epoch 16/200
1

137/137 [==============================] - 42s 308ms/step - loss: 0.6812 - accuracy: 0.5421 - val_loss: 1.7531 - val_accuracy: 0.6420
Epoch 20/200
137/137 [==============================] - ETA: 0s - loss: 0.6819 - accuracy: 0.5535
Epoch 20: val_accuracy did not improve from 0.64198
137/137 [==============================] - 18s 132ms/step - loss: 0.6819 - accuracy: 0.5535 - val_loss: 0.6811 - val_accuracy: 0.5967
Epoch 21/200
137/137 [==============================] - ETA: 0s - loss: 0.6741 - accuracy: 0.5686
Epoch 21: val_accuracy did not improve from 0.64198
137/137 [==============================] - 18s 133ms/step - loss: 0.6741 - accuracy: 0.5686 - val_loss: 0.7483 - val_accuracy: 0.5473
Epoch 22/200
137/137 [==============================] - ETA: 0s - loss: 0.6696 - accuracy: 0.5736
Epoch 22: val_accuracy did not improve from 0.64198
137/137 [==============================] - 19s 135ms/step - loss: 0.6696 - accuracy: 0.5736 - val_loss: 0.7194 - val_accuracy: 0.5535
Epoch 23/200
1

# Saving CNN extracted features

In [ ]:
CNN_OP_SIZE = 256

In [ ]:
features_reshaped, labels_reshaped = get_reshaped_features(features, labels)

In [ ]:
print(features_reshaped.shape, labels_reshaped.shape)

(6500, 128, 128, 3) (6500, 1)


In [ ]:
model = load_model(SAVED_MODEL_PATH)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 resnet50v2 (Functional)     (None, 4, 4, 2048)        23564800  
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 2, 2, 2048)       0         
 2D)                                                             
                                                                 
 dropout (Dropout)           (None, 2, 2, 2048)        0         
                                                                 
 conv2d (Conv2D)             (None, 2, 2, 512)         9437696   
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 1, 1, 512)        0         
 2D)                                                         

In [ ]:
model_outputs = model.get_layer("conv2d_1").output
child_model = tf.keras.models.Model(inputs=model.inputs, outputs=model_outputs)

In [ ]:
gen_model_output = child_model.predict(features_reshaped)

204/204 [==============================] - 8s 35ms/step


In [ ]:
gen_model_output.shape

(6500, 1, 1, 256)

In [ ]:
dim1 = features.shape[0]
dim2 = features.shape[1]
dim3 = CNN_OP_SIZE

In [ ]:
gen_model_output = gen_model_output.reshape((dim1, dim2, dim3))

In [ ]:
gen_model_output.shape

(325, 20, 256)

In [ ]:
OUTPUT_SAVE_PATH = "/content/drive/MyDrive/CDAC-Project(Personal)/numpy-arrays/Videos/Version 3/video-cnn-features({}).npz".format(SEQ_LEN)
np.savez_compressed(OUTPUT_SAVE_PATH, gen_model_output)